In [16]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [2]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [3]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████| 5/5 [02:02<00:00, 24.42s/it]


In [4]:
len(deals)

50

In [5]:
deals[44].describe()

"Title: Kohl's Lowest Prices of the Season Sale: Up to 58% off + Kohl's Cash + free shipping w/ $49\nDetails: Save up to 58% off on a sale that touches every category on the site. Plus, get $10 in Kohl's Cash for every $50 spent. Shipping adds $8.95 or is free with orders of $49 or more. Shop Now at Best Buy\nFeatures: \nURL: https://www.dealnews.com/Kohls-Lowest-Prices-of-the-Season-Sale-Up-to-58-off-Kohls-Cash-free-shipping-w-49/21777407.html?iref=rss-c196"

In [7]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}""" 

In [8]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [13]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Refurb Unlocked Samsung Galaxy S22 Ultra 256GB Phone for $299 + free shipping
Details: This is the best price we've ever seen for the 256GB version of the Galaxy S22 Ultra. It's also $41 cheaper than what you'd pay at Amazon for a refurb. It comes with a 1-year Allstate warranty. Buy Now at eBay
Features: Qualcomm SM8450 Snapdragon 8 Gen 1 8-core CPU  6.8" 3088x1440

In [19]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [20]:
result = get_recommendations()

In [ ]:

print(result)

deals=[Deal(product_description='The Unlocked Samsung Galaxy S22 Ultra is a high-performance smartphone featuring a 256GB storage capacity. It is equipped with a Qualcomm SM8450 Snapdragon 8 Gen 1 octa-core processor, ensuring smooth multitasking and app performance. The device boasts a stunning 6.8-inch 3088x1440 AMOLED touchscreen display, making visuals vibrant and clear. With a powerful 108MP back camera and a 40MP front camera, it captures beautiful photos and videos. This refurbished model comes with a one-year Allstate warranty.', price=299.0, url='https://www.dealnews.com/products/Samsung/Unlocked-Samsung-Galaxy-S22-Ultra-256-GB-Phone/382520.html?iref=rss-c142'), Deal(product_description='The Apple iPhone 16 Pro Max is a flagship smartphone offering a seamless user experience with its 256GB storage capacity. This refurbished device is in excellent condition and features a powerful camera system that includes advanced capabilities for photography and video recording. Its large d

In [30]:
from agents.scanner_agent import ScannerAgent

In [32]:
agent = ScannerAgent()
result = agent.scan()

In [34]:
print(result)

deals=[Deal(product_description='The Refurbished Unlocked Samsung Galaxy S22 Ultra features a Qualcomm Snapdragon 8 Gen 1 CPU with 8 cores, providing excellent performance for multitasking and gaming. It boasts a stunning 6.8-inch AMOLED touchscreen display with a resolution of 3088x1440 pixels, ensuring vibrant visuals. The phone also offers 256GB of storage capacity along with 8GB of RAM, perfect for storing apps, photos, and videos. Capture stunning photos with its impressive 108MP back camera and a 40MP front-facing camera for high-quality selfies.', price=299.0, url='https://www.dealnews.com/products/Samsung/Unlocked-Samsung-Galaxy-S22-Ultra-256-GB-Phone/382520.html?iref=rss-c142'), Deal(product_description="The Refurbished Unlocked Apple iPhone 16 Pro Max offers the latest technology with a sleek design. It features a spacious 256GB of storage, allowing you to store all your photos, videos, and apps without worrying about space. This device is in 'Excellent' condition and is back